In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Conv1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
import time
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.preprocessing import StandardScaler
import cv2

In [ ]:
# Load dataset (from path)
def load_gesture_path(directory):
    file_paths = []
    labels = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if len(os.listdir(category_path))!=1:
            if os.path.isdir(category_path):
                print(f"Loading category: {category}")
                for filename in os.listdir(category_path):
                    if filename.endswith('.npy'):
                        file_paths.append((category, os.path.join(category_path, filename)))
                        labels.append(category)  
    return file_paths, labels

In [ ]:
import shutil
def copy_files(file_paths, base_dir):
    for category, file_path in file_paths:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)
        shutil.copy(file_path, category_dir)

In [ ]:
# Load processed data
gesture_data_path, gesture_label = load_gesture_path('dataset-image')
print(gesture_data_path)

In [ ]:
# Split into training, testing
os.mkdir('traindata')
train_paths, test_paths, train_labels, test_labels = train_test_split(
    gesture_data_path, gesture_label, test_size=0.4, random_state=42, stratify=gesture_label
)

copy_files(train_paths, 'traindata\\train')
copy_files(test_paths, 'traindata\\test')

In [ ]:
def traindata_concate(directory):
    gesture_data = []
    labels = []
    TimeSp = 35
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            print(f"Loading category: {category}")
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if filename.endswith('.npy'):
                    data = np.load(file_path)
                    
                if data.shape == (35, 32, 32, 4):
                    gesture_data.append(data)
                    labels.append(category)
                    
                elif data.shape[0] < TimeSp:
                    print(f"Skipping {file_path}, not enough data for TimeSp")
                    continue
                
                else:
                    for i in range(100000):
                        if i + TimeSp >= data.shape[0]:
                            break
                        gesture_data.append(data[i:i+TimeSp, :, :, :])
                        labels.append(category)
    return np.array(gesture_data), np.array(labels)

In [ ]:
x_train, y_train = traindata_concate('traindata\\train')
y_train = np.repeat(y_train, 35)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
x_test, y_test = traindata_concate('traindata\\test')
y_test = np.repeat(y_test, 35)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

os.mkdir('traindata_concate_35')
np.save('traindata_concate_35/data_train.npy', x_train)
np.save('traindata_concate_35/data_test.npy', x_test)

np.save('traindata_concate_35/label_train.npy', y_train)
np.save('traindata_concate_35/label_test.npy', y_test)

In [ ]:
# Define Loss Function
def l2_loss(y_true, y_pred):
    diff = y_true - y_pred
    loss = tf.reduce_mean(tf.square(diff))
    return loss

In [ ]:
#　Define a lightweight CNN model
main_input=tf.keras.layers.Input(shape=(32,32,4), name="input_1")

x=tf.keras.layers.DepthwiseConv2D(3, strides=(2, 2), use_bias=False, name="online_cnn_fw/depthwise_conv2d_9/depthwise1")(main_input)
x=tf.keras.layers.Conv2D(32, 1, activation='relu', use_bias=False, name="online_cnn_fw/conv2d_9/Conv2D")(x)
x=tf.keras.layers.DepthwiseConv2D(3, strides=(2, 2), use_bias=False, name="online_cnn_fw/depthwise_conv2d_10/depthwise1")(x)
x=tf.keras.layers.Conv2D(64, 1, activation='relu', use_bias=False, name="online_cnn_fw/conv2d_10/Conv2D")(x)
x=tf.keras.layers.Dropout(0.5, name="dropout")(x) 
x=tf.keras.layers.DepthwiseConv2D(3, strides=(2, 2), use_bias=False, name="online_cnn_fw/depthwise_conv2d_11/depthwise1")(x)
x=tf.keras.layers.Conv2D(128, 1, activation='relu', use_bias=False, name="online_cnn_fw/conv2d_11/Conv2D1")(x)
x=tf.keras.layers.Dropout(0.5, name="dropout_1")(x) 
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.BatchNormalization(name='batch_normalization')(x)
x=tf.keras.layers.Reshape((1,128))(x)

cnnmodel=tf.keras.models.Model(inputs=main_input, outputs=x)

cnnmodel.summary()

In [ ]:
# Add a Dense classification layer with 8 output units and softmax activation
x = cnnmodel.output

predictions = Dense(11, activation='softmax')(x)

model = Model(inputs=cnnmodel.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
# Load Dataset
x_train_gesture  = np.load('traindata_concate_35\data_train.npy')
gesture_labels_train = np.load('traindata_concate_35\label_train.npy')
x_test_gesture = np.load('traindata_concate_35\data_test.npy')
gesture_labels_test = np.load('traindata_concate_35\label_test.npy')

x_train_gesture = x_train_gesture.reshape(-1, 32, 32, 4)
x_test_gesture = x_test_gesture.reshape(-1, 32, 32, 4)

gesture_labels_train = np.repeat(gesture_labels_train, 35)
gesture_labels_test = np.repeat(gesture_labels_test, 35)

print("Gesture data train shape:", np.shape(x_train_gesture))
print("Gesture labels train shape:", np.shape(gesture_labels_train))

label_encoder = LabelEncoder()
gesture_labels_train_encoded = label_encoder.fit_transform(gesture_labels_train)
gesture_labels_test_encoded = label_encoder.transform(gesture_labels_test)

gesture_labels_train_one_hot = to_categorical(gesture_labels_train_encoded).reshape(-1, 1, 11)
gesture_labels_test_one_hot = to_categorical(gesture_labels_test_encoded).reshape(-1, 1, 11)

print("Reshaped Gesture data train shape:", x_train_gesture.shape)
print("Reshaped Gesture labels train shape:", gesture_labels_train_one_hot.shape)

In [9]:
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss=l2_loss, metrics=['accuracy'])

In [8]:
def AGIprogressBar(count, total,start):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    duration=time.time()-start
    print('\r[%s] %s%s ...%s sec' % (bar, percents, '%', duration),end=' ')

In [9]:
n = len(x_train_gesture)
for i in range(n):
    j = np.random.randint(i, n)
    # Swap rows i and j for data and labels
    x_train_gesture[[i, j]], x_train_gesture[[j, i]] = x_train_gesture[[j, i]], x_train_gesture[[i, j]]
    gesture_labels_train_one_hot[[i, j]], gesture_labels_train_one_hot[[j, i]] = gesture_labels_train_one_hot[[j, i]], gesture_labels_train_one_hot[[i, j]]

In [8]:
def evaluate_data(model, x_data, y_data, batch_size, num_segments=10):
    segment_size = len(x_data) // num_segments
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size if i < num_segments - 1 else len(x_data)

        x_segment = x_data[start_idx:end_idx]
        y_segment = y_data[start_idx:end_idx]

        result = model.evaluate(x_segment, y_segment, batch_size=batch_size, verbose=0)
        segment_loss = result[0]
        predictions = model.predict(x_segment, batch_size=batch_size, verbose=0)

        total_loss += segment_loss * len(y_segment)

        correct_predictions += np.sum(np.argmax(predictions, axis=2) == np.argmax(y_segment, axis=2))
        total_samples += len(y_segment)

    train_loss = total_loss / total_samples
    train_acc = correct_predictions / total_samples

    return train_loss, train_acc

In [ ]:
Batch = 8960
epochs = 1000
segment_count = 25
rec = []
st = time.time()
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for ep in range(epochs):
    print(f'EP: {ep + 1}')
    segment_size = len(x_train_gesture) // segment_count

    for seg in range(segment_count):
        start_idx = seg * segment_size
        end_idx = start_idx + segment_size

        x_segment = x_train_gesture[start_idx:end_idx]
        y_segment = gesture_labels_train_one_hot[start_idx:end_idx]

        indices = np.arange(len(x_segment))
        np.random.shuffle(indices)
        
        # Apply the shuffled indices to the segments
        x_segment = x_segment[indices]
        y_segment = y_segment[indices]

        for i in range(len(x_segment) // Batch):
            AGIprogressBar(i, len(x_segment) // Batch, st)
            x_batch = x_segment[i * Batch:(i + 1) * Batch]
            y_batch = y_segment[i * Batch:(i + 1) * Batch]

            model.train_on_batch(x_batch, y_batch)

    train_loss, train_acc = evaluate_data(
        model=model,
        x_data=x_train_gesture,
        y_data=gesture_labels_train_one_hot,
        batch_size=3200,
        num_segments=700
    )

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    print(' ')
    print(f'Epoch {ep + 1} Training Loss = {train_loss}')
    print(f'Epoch {ep + 1} Training ACC = {train_acc}')

    val_loss, val_acc = evaluate_data(
        model=model,
        x_data=x_test_gesture,
        y_data=gesture_labels_test_one_hot,
        batch_size=3200,
        num_segments=700
    )

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)


    print(f'Epoch {ep + 1} Validation Loss = {val_loss}')
    print(f'Epoch {ep + 1} Validation ACC = {val_acc}')

    os.makedirs('saved_model_our_cnn', exist_ok=True)
    if (ep + 1) % 1 == 0:
        model.save(f'saved_model_our_cnn/epoch_{ep + 1}.h5')
        print(f'Model saved at epoch {ep + 1}')
    tf.keras.backend.clear_session()
    gc.collect()
    model = load_model(f'saved_model_our_cnn/epoch_{ep + 1}.h5', custom_objects={'l2_loss': l2_loss})


best_epoch_accuracy = val_accuracies.index(max(val_accuracies))
best_val_accuracy = val_accuracies[best_epoch_accuracy]
print(f"The best epoch based on validation accuracy is: {best_epoch_accuracy + 1}, with accuracy: {best_val_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()


In [ ]:
import re
import matplotlib.pyplot as plt

epoch_data = {}

with open('log.txt', 'r') as f:
    for line in f:
        line = line.strip()
        match = re.search(r'Epoch (\d+) Training Loss = (\d+\.\d+)', line)
        if match:
            epoch = int(match.group(1))
            loss = float(match.group(2))
            if epoch not in epoch_data:
                epoch_data[epoch] = {'train_loss': None, 'train_acc': None, 'val_loss': None, 'val_acc': None}
            epoch_data[epoch]['train_loss'] = loss
            continue
        match = re.search(r'Epoch (\d+) Training ACC = (\d+\.\d+)', line)
        if match:
            epoch = int(match.group(1))
            acc = float(match.group(2))
            if epoch not in epoch_data:
                epoch_data[epoch] = {'train_loss': None, 'train_acc': None, 'val_loss': None, 'val_acc': None}
            epoch_data[epoch]['train_acc'] = acc
            continue
        match = re.search(r'Epoch (\d+) Validation Loss = (\d+\.\d+)', line)
        if match:
            epoch = int(match.group(1))
            val_loss = float(match.group(2))
            if epoch not in epoch_data:
                epoch_data[epoch] = {'train_loss': None, 'train_acc': None, 'val_loss': None, 'val_acc': None}
            epoch_data[epoch]['val_loss'] = val_loss
            continue
        match = re.search(r'Epoch (\d+) Validation ACC = (\d+\.\d+)', line)
        if match:
            epoch = int(match.group(1))
            val_acc = float(match.group(2))
            if epoch not in epoch_data:
                epoch_data[epoch] = {'train_loss': None, 'train_acc': None, 'val_loss': None, 'val_acc': None}
            epoch_data[epoch]['val_acc'] = val_acc
            continue

epochs = sorted(epoch_data.keys())
train_losses = [epoch_data[e]['train_loss'] for e in epochs]
train_accuracies = [epoch_data[e]['train_acc'] for e in epochs]
val_losses = [epoch_data[e]['val_loss'] for e in epochs]
val_accuracies = [epoch_data[e]['val_acc'] for e in epochs]

best_epoch_accuracy = val_accuracies.index(max(val_accuracies))
best_val_accuracy = val_accuracies[best_epoch_accuracy]
print(f"The best epoch based on validation accuracy is: {best_epoch_accuracy + 1}, with accuracy: {best_val_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [3]:
model = load_model(f'saved_model_our_cnn/epoch_100.h5', custom_objects={'l2_loss': l2_loss})

In [ ]:
gesture_labels_train = np.load('traindata_concate_35\label_train.npy')
x_test_gesture = np.load('traindata_concate_35\data_test.npy')

gesture_labels_test = np.load('traindata_concate_35\label_test.npy')

x_test_gesture = x_test_gesture.reshape(-1, 32, 32, 4)

gesture_labels_train = np.repeat(gesture_labels_train, 35)
gesture_labels_test = np.repeat(gesture_labels_test, 35)

print("Gesture labels train shape:", np.shape(gesture_labels_train))

label_encoder = LabelEncoder()
gesture_labels_train_encoded = label_encoder.fit_transform(gesture_labels_train)
gesture_labels_test_encoded = label_encoder.transform(gesture_labels_test)

gesture_labels_train_one_hot = to_categorical(gesture_labels_train_encoded).reshape(-1, 1, 11)
gesture_labels_test_one_hot = to_categorical(gesture_labels_test_encoded).reshape(-1, 1, 11)

print("Reshaped Gesture labels train shape:", gesture_labels_train_one_hot.shape)

In [12]:
# hook BN Layer
hook=[]
id=[10]
for i in range(len(id)):
  hook.append(model.layers[id[i]].output)
ModelExtract = Model(inputs=model.input, outputs=hook)


In [ ]:
import os
import numpy as np

def process_data_in_groups(data, model, save_dir, prefix, num_segments=15, group_size=5):
    """
    Splits the data into `num_segments` parts, processes them in groups of `group_size`,
    saves intermediate group predictions to disk, then concatenates them to form final predictions.
    
    Parameters:
      data (np.array): The input data (e.g., x_train_gesture or x_test_gesture).
      model (keras.Model): The model to use for prediction.
      save_dir (str): Directory to save the intermediate and final prediction files.
      prefix (str): File name prefix, e.g., 'train' or 'test'.
      num_segments (int): Total number of segments to split the data into.
      group_size (int): Process predictions in groups of this many segments.
      
    Returns:
      np.array: The final concatenated predictions.
    """
    os.makedirs(save_dir, exist_ok=True)
    data_segments = np.array_split(data, num_segments)
    group_file_paths = []
    
    for group_idx in range(0, num_segments, group_size):
        current_segment_files = []
        for i in range(group_idx, min(group_idx + group_size, num_segments)):
            preds = model.predict(data_segments[i])
            seg_file_name = f'{prefix}_segment_{i}.npy'
            seg_file_path = os.path.join(save_dir, seg_file_name)
            np.save(seg_file_path, preds)
            current_segment_files.append(seg_file_path)

        group_preds = np.concatenate([np.load(file) for file in current_segment_files], axis=0)
        group_file_name = f'{prefix}_group_{group_idx // group_size}.npy'
        group_file_path = os.path.join(save_dir, group_file_name)
        np.save(group_file_path, group_preds)
        group_file_paths.append(group_file_path)
        
        for file in current_segment_files:
            os.remove(file)
        tf.keras.backend.clear_session()
        gc.collect()
        print(f"[{prefix}] Processed group {group_idx // group_size + 1} (segments {group_idx} to {min(group_idx+group_size, num_segments)-1}).")

    final_preds = np.concatenate([np.load(file) for file in group_file_paths], axis=0)
    final_file_path = os.path.join(save_dir, f'data_{prefix}.npy')
    np.save(final_file_path, final_preds)
    print(f"[{prefix}] Final predictions saved to {final_file_path}")
    
    return final_preds

save_directory = 'Stage1_LSTM'

train_predictions = process_data_in_groups(
    data=x_train_gesture,
    model=ModelExtract,
    save_dir=save_directory,
    prefix='train',
    num_segments=15,
    group_size=5
)

test_predictions = process_data_in_groups(
    data=x_test_gesture,
    model=ModelExtract,
    save_dir=save_directory,
    prefix='test',
    num_segments=15,
    group_size=5
)
